In [2]:
"""
此程序用于连接到当前打开的SAP2000模型
"""

import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import comtypes.client
import time
from collections import defaultdict # 用于存储楼层信息, 方便创建刚性隔板

def connect_to_sap2000():
    """
    连接到当前打开的SAP2000实例
    
    返回:
        成功连接时返回SAP2000模型对象，否则返回None
    """
    try:
        print("尝试连接到SAP2000...")
        
        # 创建SAP2000 API帮助对象
        helper = comtypes.client.CreateObject('SAP2000v1.Helper')
        helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
        
        # 获取当前打开的SAP2000实例
        mySapObject = helper.GetObject("CSI.SAP2000.API.SapObject")
        if mySapObject is None:
            print("找不到打开的SAP2000实例，尝试启动新实例...")
            
            # 启动新的SAP2000实例
            mySapObject = helper.CreateObject("CSI.SAP2000.API.SapObject")
            mySapObject.ApplicationStart()
            
        # 获取激活的模型
        model = mySapObject.SapModel
        
        # 检查是否已打开模型
        file_path = model.GetModelFilename()
        if not file_path:
            print("警告: SAP2000中未打开模型。请先打开一个模型文件。")
        else:    
            print(f"成功连接到SAP2000！当前模型文件: {file_path}")
        
        return model
    
    except Exception as e:
        print(f"连接到SAP2000时出错: {e}")
        return None

def get_model_info(model):
    """
    获取并打印SAP2000模型的详细信息
    
    参数:
        model: SAP2000模型对象
    """
    try:
        print("\n" + "=" * 40)
        print("SAP2000模型详细信息:")
        print("=" * 40)
        
        # 获取所有节点信息
        _, point_names, _, _, _, _, _ = model.PointObj.GetAllPoints()
        print(f"\n节点总数: {len(point_names)}")
        print(f"前10个节点: {point_names[:10] if len(point_names) > 10 else point_names}")
        
        # 获取所有框架元素信息
        _, frame_names, _, _, _ = model.FrameObj.GetAllFrames()
        print(f"\n框架元素总数: {len(frame_names)}")
        print(f"前10个框架元素: {frame_names[:10] if len(frame_names) > 10 else frame_names}")
        
        # 获取所有载荷模式
        _, load_patterns = model.LoadPatterns.GetNameList()
        print(f"\n载荷模式总数: {len(load_patterns)}")
        print(f"载荷模式: {load_patterns}")
        
        # 获取所有载荷组合
        _, load_combos = model.RespCombo.GetNameList()
        print(f"\n载荷组合总数: {len(load_combos)}")
        print(f"载荷组合: {load_combos}")
        
        # 获取所有隔板信息
        _, diaphragm_names = model.AreaObj.GetNameListDiaphragm()
        print(f"\n隔板总数: {len(diaphragm_names)}")
        print(f"隔板: {diaphragm_names}")
        
        print("\n" + "=" * 40)
        
        return True
    except Exception as e:
        print(f"获取模型信息时出错: {e}")
        return False

def add_diaphragms(model, target_elevations=None, tolerance=0.01):
    """
    在SAP2000模型中创建刚性隔板
    
    参数:
        model: SAP2000模型对象
        target_elevations: 指定的楼层标高列表，如果为None则使用所有标高（可选）
        tolerance: 楼层标高容差（可选）

    返回:
        成功创建的隔板名称列表，如果失败则返回空列表
    """
    try:        
        # 获取所有节点的Z坐标（即楼层标高）
        node_info = []  # 用于保存所有节点名称和坐标
        node_z_coords = defaultdict(list)
        constraint_names = []  # 用于存储创建的约束名称

        # 获取模型中的楼层信息
        number_of_points, point_names, ret = model.PointObj.GetNameList()
       
        for point_name in point_names:
            [x, y, z, ret] = model.PointObj.GetCoordCartesian(point_name)
            node_info.append({"name": point_name, "x": x, "y": y, "z": z})
        
        # 如果指定了目标标高，则筛选节点
        if target_elevations is not None:
            print(f"按照指定标高筛选节点：{target_elevations}")
            filtered_nodes = []
            for node in node_info:
                # 检查节点是否在任意目标标高附近
                for elevation in target_elevations:
                    if abs(node["z"] -elevation) <= tolerance:
                        filtered_nodes.append(node)
                        # 将节点按照最接近的标高值进行分组
                        closest_z = min(target_elevations, key=lambda e: abs(e - node["z"]))
                        node_z_coords[closest_z].append(node["name"])
                        break
            print(f"筛选后剩余{len(filtered_nodes)}个节点")
        else:
            print("未指定目标标高，使用所有节点")
            for node in node_info:
                node_z_coords[round(node["z"], 3)].append(node["name"])  # 使用Z坐标分组，保留3位小数

        # 保存节点信息到CVS文件
        df = pd.DataFrame(node_info)
        csv_path = os.path.join(os.getcwd(), "node_coordinates.csv")
        df.to_csv(csv_path, index=False)
        print(f"节点坐标已保存到: {csv_path}")            

        # 为每一组节点设置刚性隔板约束
        print(f"将为以下Z坐标创建刚性隔板约束: {list(node_z_coords.keys())}")

        for z_value, nodes in node_z_coords.items():
            constraint_name = f"Diaphragm_Z_{z_value}"  # 根据Z值生成约束名称
            
            # 首先定义刚性隔板的名称
            ret = model.ConstraintDef.SetDiaphragm(constraint_name, 3, "Global")  # 设置刚性隔板约束

            # print(f"创建刚性隔板约束: {constraint_name}，包含节点: {nodes}")
            constraint_names.append(constraint_name)  # 保存约束名称

            # 设置刚性隔板约束
            # model.ConstraintDef.SetDiaphragm(constraint_name, nodes, "Global")  # 设置刚性隔板约束
            for i in nodes:
                model.PointObj.SetConstraint(i,constraint_name)

        print("刚性隔板约束设置完成！")
        return constraint_names, node_z_coords  # 返回创建的约束名称列表

    except Exception as e:
        print(f"创建刚性隔板时出错: {e}")
        import traceback
        traceback.print_exc()
        return []

def add_wind_time_history_load(model, diaphragm_constraints, node_z_coords, wind_time_history_file=None):
    """
    在每个刚性隔板的中心点添加风荷载时程曲线
    
    参数:
        model: SAP2000模型对象
        diaphragm_constraints: 刚性隔板约束名称列表
        node_z_coords: 节点Z坐标字典
        wind_time_history_file: 风荷载时程函数文件路径（如果为None，则使用默认函数）

    返回:
        成功添加荷载的数量
    """
    print("\n开始添加风荷载时程曲线...")
    
    # 步骤1：为每个刚性隔板找到中心点
    diaphragm_centers = {}
    success_count = 0
    
    for constraint_name in diaphragm_constraints:
        # 获取约束中的所有节点
        constraint_points = []
        point_names = node_z_coords.get(round(float(constraint_name.split('_')[-1]), 3), [])
        
        # 遍历所有节点，找出使用该约束的节点
        # _, point_names, _ = model.PointObj.GetNameList()
        for point_name in point_names:
            # 获取节点坐标
            [x, y, z, _] = model.PointObj.GetCoordCartesian(point_name)
            constraint_points.append({"name": point_name, "x": x, "y": y, "z": z})
        if not constraint_points:
            print(f"警告：隔板 {constraint_name} 没有关联节点，跳过")
            continue
        print(f"隔板 {constraint_name} 包含 {len(constraint_points)} 个节点")

        # 计算中心点坐标
        avg_x = sum(point["x"] for point in constraint_points) / len(constraint_points)
        avg_y = sum(point["y"] for point in constraint_points) / len(constraint_points)
        avg_z = sum(point["z"] for point in constraint_points) / len(constraint_points)
        
        # 步骤2：在中心点创建新节点（或使用最接近中心的现有节点）
        # 查找最接近中心的节点
        closest_point = min(constraint_points, 
                            key=lambda p: ((p["x"]-avg_x)**2 + (p["y"]-avg_y)**2)**0.5)
                            
        center_point_name = f"WIND_CENTER_{constraint_name}"
        
        # 检查是否需要创建新节点（如果中心点附近没有现有节点）
        if ((closest_point["x"]-avg_x)**2 + (closest_point["y"]-avg_y)**2)**0.5 > 1:  # 如果最近点距离中心超过1mm
            # 创建新节点
            ret = model.PointObj.AddCartesian(avg_x, avg_y, avg_z, center_point_name)
            if ret[-1] != 0:
                print(f"创建中心点失败：{constraint_name}，使用最近节点")
                center_point_name = closest_point["name"]
            else:
                # 将新节点添加到隔板约束中
                model.PointObj.SetConstraint(center_point_name, constraint_name)
                print(f"在隔板 {constraint_name} 中创建了中心点: {center_point_name}")
        else:
            # 使用最近的现有节点
            center_point_name = closest_point["name"]
            print(f"使用隔板 {constraint_name} 中的最近点作为中心: {center_point_name}")
            
        diaphragm_centers[constraint_name] = {
            "point_name": center_point_name, 
            "x": avg_x, 
            "y": avg_y, 
            "z": avg_z
        }

    # 读取CSV文件中的数据
    df = pd.read_csv(wind_time_history_file, header=None)
    fs = 8.3227

    # 生成时间序列
    MyTime  = [i/fs for i in range(1, len(df))]
    print(f"时程数据共有 {len(df)} 行，采样频率 {fs}Hz")
    print(f"时间序列前5个值: {MyTime[:5]}")

    # 限值缝隙行数以加快测试速度（实际分析请移除次限制）
    num_columns = df.shape[1]
    # num_rows = df.shape[0]
    num_rows = 16000 # 限制行数为16000行，便于测试
    MyTime = MyTime[:num_rows]

    # 删除可能存在的旧荷载模式和时程函数
    try:
        # 删除旧的荷载模式
        existing_patterns = []
        try:
            _, existing_patterns = model.LoadPatterns.GetNameList()
        except:
            pass
            
        for pattern in existing_patterns:
            if pattern.startswith("Wind_"):
                try:
                    model.LoadPatterns.Delete(pattern)
                    print(f"已删除荷载模式: {pattern}")
                except:
                    pass
        
        # 删除旧的时程工况
        try:
            model.LoadCases.Delete("WIND_TIME_HISTORY")
            print("已删除旧的时程工况")
        except:
            pass
            
        # 删除旧的时程函数
        try:
            _, func_names = model.Func.GetNameList()
            for func in func_names:
                if func.startswith("WIND_"):
                    model.Func.Delete(func)
            print("已删除旧的时程函数")
        except:
            pass
    except Exception as e:
        print(f"清理旧数据时出错: {e}")

    # 创建统一的时程工况
    unified_case_name = "Wind_time_history"
    ret = model.LoadCases.DirHistLinear.SetCase(unified_case_name)
    if ret != 0:
        print(f"创建时程工况失败: {unified_case_name}")
        return 0
    
    print(f"创建统一的时程工况: {unified_case_name}")

    # 存储所有荷载参数，用于后续批量添加到时程工况
    load_types = []      # 荷载类型
    load_patterns = []   # 荷载模式名称
    func_names = []      # 时程函数名称
    scales = []          # 比例系数
    tfactors = []        # 时间比例因子
    delays = []          # 时间延迟
    coord_systems = []   # 坐标系
    angles = []          # 角度

    # 步骤5：为每个隔板中心添加风荷载
    col_idx = 0
    # 创建响应组合
    ret = model.RespCombo.Delete("Combo2")
    ret = model.RespCombo.Add("Combo2", 0)
    for constraint_name, center_info in diaphragm_centers.items():
        # point_name = center_info["point_name"]
        point_name = "195"
        
        print(f"首先删除隔板 {constraint_name} 中心点 {point_name} 的风荷载")
        ret = model.PointObj.DeleteLoadForce(point_name, "Wind")

        col_data = df[col_idx].values.tolist() # 提取当前列的风荷载数据
        Wind_func_name = f"Wind_x_{col_idx + 1}" # 生成风荷载时程函数名称
        ret = model.Func.FuncTH.SetUser(Wind_func_name, len(MyTime), MyTime, col_data)  # 从文件创建风荷载时程函数
        
        LoadPatternName = Wind_func_name # 创建荷载模式
        ret = model.LoadPatterns.Add(LoadPatternName, 6, 0, True)

        # 在节点上施加荷载
        x_force = 1  # X方向风荷载
        ret = model.PointObj.SetLoadForce(point_name, LoadPatternName, [x_force, 0, 0, 0, 0, 0], True, "Global", 0)
        print(f"在隔板 {constraint_name} 中心点 {point_name} 添加风荷载X方向风荷载时程函数: {Wind_func_name}")

        # 将荷载参数添加到列表
        # X方向
        load_types.append("Load")
        load_patterns.append(LoadPatternName)
        func_names.append(Wind_func_name)
        scales.append(1.0)
        tfactors.append(1.0)
        delays.append(0.0)
        coord_systems.append("Global")
        angles.append(0.0)

        col_idx += 1
        # 提取当前列的风荷载数据
        col_data = df[col_idx].values.tolist()
        Wind_func_name = f"Wind_y_{col_idx + 1}"
        ret = model.Func.FuncTH.SetUser(Wind_func_name, len(MyTime), MyTime, col_data)  # 从文件创建风荷载时程函数
        # 创建荷载模式
        LoadPatternName = Wind_func_name
        ret = model.LoadPatterns.Add(LoadPatternName, 6, 0, True)
        # 在节点上施加荷载
        y_force = 1  # Y方向风荷载
        ret = model.PointObj.SetLoadForce(point_name, LoadPatternName, [0, y_force, 0, 0, 0, 0], True, "Global", 0)
        print(f"在隔板 {constraint_name} 中心点 {point_name} 添加风荷载Y方向风荷载时程函数: {Wind_func_name}")

        # 将荷载参数添加到列表
        # Y方向
        load_types.append("Load")
        load_patterns.append(LoadPatternName)
        func_names.append(Wind_func_name)
        scales.append(1.0)
        tfactors.append(1.0)
        delays.append(0.0)
        coord_systems.append("Global")
        angles.append(0.0)


        col_idx += 1
        # 提取当前列的风荷载数据
        col_data = df[col_idx].values.tolist()
        Wind_func_name = f"Wind_z_{col_idx + 1}"
        ret = model.Func.FuncTH.SetUser(Wind_func_name, len(MyTime), MyTime, col_data)  # 从文件创建风荷载时程函数
        # 创建荷载模式
        LoadPatternName = Wind_func_name
        ret = model.LoadPatterns.Add(LoadPatternName, 6, 0, True)
        # 在节点上施加荷载
        z_force = 1  # Z方向风荷载
        ret = model.PointObj.SetLoadForce(point_name, LoadPatternName, [0, 0, 0, 0, 0, z_force], True, "Global", 0)
        print(f"在隔板 {constraint_name} 中心点 {point_name} 添加风荷载Z方向风荷载时程函数: {Wind_func_name}")
        
        # 将荷载参数添加到列表
        # Z方向
        load_types.append("Load")
        load_patterns.append(LoadPatternName)
        func_names.append(Wind_func_name)
        scales.append(1.0)
        tfactors.append(1.0)
        delays.append(0.0)
        coord_systems.append("Global")
        angles.append(0.0)

        # 将工况添加到响应组合
        ret = model.RespCombo.SetCaseList("Combo2", 0, LoadPatternName, 1)
        col_idx += 1

        if col_idx >= 3:  # 限制测试前3列数据
            break  # 仅测试前3列数据

    # 步骤7：将所有荷载关联到统一时程工况
    num_loads = len(load_patterns)
    print(f"将 {num_loads} 个荷载关联到统一时程工况 {unified_case_name}")

    # 提交所有荷载到统一时程工况
    ret = model.LoadCases.DirHistLinear.SetLoads(
        unified_case_name,
        num_loads,
        load_types,
        load_patterns,
        func_names,
        scales,
        tfactors,
        delays,
        coord_systems,
        angles
    )
    ret = model.LoadCases.DirHistLinear.SetTimeIntegration(unified_case_name, 1, 0, 0.5, 0.25, 0, 0)
    ret = model.LoadCases.DirHistLinear.SetTimeStep(unified_case_name, num_rows, 1/fs)

    if ret != 0:
        print(f"将荷载关联到时程工况失败, 错误码: {ret}")
    else:
        print(f"成功将所有荷载关联到时程工况 {unified_case_name}")

    # 设置运行工况
    ret = model.Analyze.SetRunCaseFlag(unified_case_name, True)
    
    print(f"风荷载时程曲线添加完成，共添加了 {col_idx} 个荷载")

    return col_idx, diaphragm_centers
        
def get_node_displacement_history(model, node_name, load_case="Wind_time_history", output_file=None):
    """
    获取指定节点在指定荷载工况下的位移响应时程
    
    参数:
        model: SAP2000模型对象
        node_name: 要获取位移的节点名称
        load_case: 荷载工况名称，默认为"Wind_time_history"
        output_file: 输出CSV文件路径，如果指定则将结果保存到CSV文件
        
    返回:
        成功时返回元组(时间列表, [X位移列表, Y位移列表, Z位移列表, X旋转列表, Y旋转列表, Z旋转列表])
        失败时返回(None, None)
    """
    try:
        print(f"\n获取节点 {node_name} 在 {load_case} 工况下的位移响应时程...")
        
        # 检查节点是否存在
        ret = model.PointObj.GetNameList()
        if ret[0] == 0 and node_name not in ret[1]:
            print(f"错误: 节点 {node_name} 不存在")
            return None, None
            
        # 检查荷载工况是否存在
        ret = model.LoadCases.GetNameList()
        if ret[0] == 0 and load_case not in ret[1]:
            print(f"错误: 荷载工况 {load_case} 不存在")
            return None, None
            
        # 获取时间步长信息
        ret = model.LoadCases.DirHistLinear.GetTimeStep(load_case)
        if ret[-1] != 0:
            print(f"获取时间步长失败，错误码: {ret[0]}")
            return None, None
            
        num_steps = ret[0]
        time_step = ret[1]
        print(f"时程分析包含 {num_steps} 个时间步，步长为 {time_step} 秒")

        ret = model.Results.Setup.DeselectAllCasesAndCombosForOutput()
        ret = model.Results.Setup.SetCaseSelectedForOutput(load_case)

        # 获取位移结果（注意正确的方法名）
        GroupElm = 0
        NumberResults = 0
        Obj = []
        Elm = []
        LoadCase = [load_case]
        StepType = ["Time"]
        StepNum = []
        U1, U2, U3, R1, R2, R3 = [], [], [], [], [], []
        
        [NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3, ret] = \
        model.Results.JointDispl(
            node_name, 
            GroupElm, 
            NumberResults, 
            Obj, 
            Elm, 
            LoadCase, 
            StepType, 
            StepNum,
            U1, U2, U3, R1, R2, R3 
        )
        
        ux_list = U1  # X方向位移
        uy_list = U2  # Y方向位移
        uz_list = U3  # Z方向位移
        rx_list = R1  # X方向旋转
        ry_list = R2  # Y方向旋转
        rz_list = R3  # Z方向旋转

        # 汇总位移结果
        displacement_results = [ux_list, uy_list, uz_list, rx_list, ry_list, rz_list]
        time_points = [i * time_step for i in range(num_steps+1)]  # 生成时间点列表

        # 如果指定了输出文件，保存结果到CSV
        if output_file:
            try:
                import pandas as pd
                import os
                
                # 创建DataFrame
                df = pd.DataFrame({                    
                    "UX": ux_list,
                    "UY": uy_list,
                    "UZ": uz_list,
                    "RX": rx_list,
                    "RY": ry_list,
                    "RZ": rz_list
                })
                
                # 确保输出目录存在
                output_dir = os.path.dirname(output_file)
                if output_dir and not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                    
                # 保存到CSV文件
                df.to_csv(output_file, index=False)
                print(f"位移响应时程已保存至: {output_file}")
                
                # 输出简单统计信息
                print("\n位移响应统计:")
                print(f"X方向最大位移: {max(ux_list, key=abs):.6f} mm")
                print(f"Y方向最大位移: {max(uy_list, key=abs):.6f} mm")
                print(f"Z方向最大位移: {max(uz_list, key=abs):.6f} mm")
                
            except Exception as e:
                print(f"保存CSV文件时出错: {e}")
                
        return time_points, displacement_results
        
    except Exception as e:
        print(f"获取节点位移响应时程时出错: {e}")
        import traceback
        traceback.print_exc()
        return None, None

def main():
    print("=" * 80)
    print("SAP2000模型连接程序")
    print("=" * 80)
    
    # 连接到当前打开的SAP2000实例
    model = connect_to_sap2000()
    print(model)
    if model is None:
        print("无法连接到SAP2000，程序终止")
        return
    
    # [2] 锁定/解锁模型
    locked = model.GetModelIsLocked()
    if locked:
        model.SetModelIsLocked(False)
        print("模型已解锁")
    else:
        # model.SetModelIsLocked(True)
        print("模型未锁定")

    # 创建刚性隔板
    print("开始创建刚性隔板...")
    # 指定要创建刚性隔板的楼层标高列表
    target_elevations = [6000, 10500, 15000, 19500, 23100, 26700, 30300, 33900, 37500, 41100, 44700, 48300, 51900, 55500, 
                         59100, 62700, 66300, 69900, 73500, 77100, 80700, 84300, 87900, 91500, 95100, 98700, 102300, 
                         105900, 109500, 113100, 116700, 120300, 123900, 127500, 131100, 134700, 138300, 141900, 145500, 
                         149100, 152700, 156300, 159900, 163500, 167100, 170700, 174300, 177900, 181500, 185100, 188700, 
                         192300, 196150, 200000]

    diaphragm_constraints, node_z_coords = add_diaphragms(model, target_elevations=target_elevations, tolerance=10)
    if diaphragm_constraints:
        print(f"成功创建刚性隔板: {diaphragm_constraints}")
    else:
        print("创建刚性隔板失败")

    # 添加风荷载时程曲线
    # 使用自定义风荷载时程文件
    wind_file_path = "D:\\MyFiles\\SAP2000\\code\\WindloadTimes\\Model2_10yr_000.csv"
    wind_load_count, diaphragm_centers = add_wind_time_history_load(model, diaphragm_constraints, node_z_coords, wind_time_history_file=wind_file_path)
    if wind_load_count > 0:
        print(f"成功添加 {wind_load_count} 个风荷载时程曲线")
    else:
        print("添加风荷载时程曲线失败")
        
    # [4] 运行分析
    print("开启多线程求解器...")
    ret = model.Analyze.SetSolverOption_1(2,0,True)
    print("正在运行分析...")
    ret = model.Analyze.RunAnalysis()
    if ret == 0:
        print("分析已成功完成")
    else:
        print(f"分析失败，返回代码: {ret}")

    # 获取节点位移响应时程
    center_nodes = []  # 收集所有风荷载中心点
    for constraint_name, info in diaphragm_centers.items():
        center_nodes.append(info["point_name"])
    
    if center_nodes:
        # 选择第一个中心点进行分析
        # target_node = center_nodes[-1]
        target_node = "54000071"
        print(f"\n分析中心点 {target_node} 的位移响应...")
        
        # 输出文件路径
        output_path = "D:\\MyFiles\\SAP2000\\results\\node_displacement.csv"
        
        # 获取位移响应时程
        times, displacements = get_node_displacement_history(
            model,
            target_node,
            load_case="Wind_time_history", 
            output_file=output_path
        )
        
        if times and displacements:
            print(f"成功获取 {len(times)} 个时间步的位移数据")
        else:
            print("获取位移响应失败")
            
        # 获取最上层中心点的位移响应
        top_floor = sorted(diaphragm_centers.keys(), key=lambda x: float(x.split('_')[-1]))[-1]
        top_node = diaphragm_centers[top_floor]["point_name"]
        
        top_output_path = "D:\\MyFiles\\SAP2000\\results\\top_node_displacement.csv"
        
        print(f"\n分析顶层中心点 {top_node} 的位移响应...")
        top_times, top_displacements = get_node_displacement_history(
            model,
            top_node,
            load_case="Wind_time_history", 
            output_file=top_output_path
        )
        
        if top_times and top_displacements:
            print(f"成功获取顶层节点 {top_node} 的 {len(top_times)} 个时间步的位移数据")
    print("=" * 80)

if __name__ == "__main__":
    main()

SAP2000模型连接程序
尝试连接到SAP2000...
成功连接到SAP2000！当前模型文件: D:\MyFiles\SAP2000\1.1model2\1栋-QG.sdb
<POINTER(cSapModel) ptr=0x15138a2faa0 at 15136310a50>
模型未锁定
开始创建刚性隔板...
按照指定标高筛选节点：[6000, 10500, 15000, 19500, 23100, 26700, 30300, 33900, 37500, 41100, 44700, 48300, 51900, 55500, 59100, 62700, 66300, 69900, 73500, 77100, 80700, 84300, 87900, 91500, 95100, 98700, 102300, 105900, 109500, 113100, 116700, 120300, 123900, 127500, 131100, 134700, 138300, 141900, 145500, 149100, 152700, 156300, 159900, 163500, 167100, 170700, 174300, 177900, 181500, 185100, 188700, 192300, 196150, 200000]
筛选后剩余15558个节点
节点坐标已保存到: d:\MyFiles\SAP2000\code\node_coordinates.csv
将为以下Z坐标创建刚性隔板约束: [6000, 10500, 15000, 19500, 23100, 26700, 30300, 33900, 37500, 41100, 44700, 48300, 51900, 55500, 59100, 62700, 66300, 69900, 73500, 77100, 80700, 84300, 87900, 91500, 95100, 98700, 102300, 105900, 109500, 113100, 116700, 120300, 123900, 127500, 131100, 134700, 138300, 141900, 145500, 149100, 152700, 156300, 159900, 163500, 167100, 1

In [15]:
# 在新的单元格中运行以下代码
import os
import sys
import pandas as pd
import numpy as np
import comtypes.client
import matplotlib.pyplot as plt

def reconnect_to_sap2000():
    """
    重新连接到当前打开的SAP2000实例
    """
    try:
        print("重新连接到SAP2000...")
        
        # 创建SAP2000 API帮助对象
        helper = comtypes.client.CreateObject('SAP2000v1.Helper')
        helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
        
        # 获取当前打开的SAP2000实例
        mySapObject = helper.GetObject("CSI.SAP2000.API.SapObject")
        if mySapObject is None:
            print("找不到打开的SAP2000实例")
            return None
            
        # 获取激活的模型
        model = mySapObject.SapModel
        
        # 检查是否已打开模型
        file_path = model.GetModelFilename()
        if not file_path:
            print("警告: SAP2000中未打开模型")
        else:    
            print(f"成功重新连接到SAP2000！当前模型文件: {file_path}")
        
        return model
    
    except Exception as e:
        print(f"重新连接到SAP2000时出错: {e}")
        return None

def get_displacement_data(model, node_name, load_case="Wind_time_history"):
    """
    获取指定节点的位移时程数据
    
    参数:
        model: SAP2000模型对象
        node_name: 节点名称
        load_case: 荷载工况名称
        
    返回:
        times: 时间列表
        displacements: 位移数据字典
    """
    try:
        print(f"\n正在获取节点 {node_name} 的位移数据...")
        
        # 设置输出工况
        ret = model.Results.Setup.DeselectAllCasesAndCombosForOutput()
        ret = model.Results.Setup.SetCaseSelectedForOutput(load_case)
        
        # 获取时间步长信息
        ret = model.LoadCases.DirHistLinear.GetTimeStep(load_case)
        if ret[-1] != 0:
            print(f"获取时间步长失败，错误码: {ret[-1]}")
            return None, None
            
        num_steps = ret[0]
        time_step = ret[1]
        print(f"时程分析包含 {num_steps} 个时间步，步长为 {time_step} 秒")

        # 初始化变量
        GroupElm = 0
        NumberResults = 0
        Obj = []
        Elm = []
        LoadCase = ["Wind_time_history"]
        StepType = ["Time"]
        StepNum = []
        U1, U2, U3, R1, R2, R3 = [], [], [], [], [], []
        
        # 获取位移结果
        [NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3, ret] = \
        model.Results.JointDispl(
            node_name, 
            GroupElm, 
            NumberResults, 
            Obj, 
            Elm, 
            LoadCase, 
            StepType, 
            StepNum,
            U1, U2, U3, R1, R2, R3 
        )
        
        if ret != 0:
            print(f"获取位移结果失败，错误码: {ret}")
            return None, None
            
        # 生成时间序列
        times = [i * time_step for i in range(len(U1))]
        
        # 整理位移数据
        displacements = {
            'UX': U1,      # X方向位移 (mm)
            'UY': U2,      # Y方向位移 (mm)  
            'UZ': U3,      # Z方向位移 (mm)
            'RX': R1,      # X方向旋转 (rad)
            'RY': R2,      # Y方向旋转 (rad)
            'RZ': R3       # Z方向旋转 (rad)
        }
        
        print(f"成功获取 {len(times)} 个时间步的位移数据")
        
        # 显示统计信息
        print("\n位移统计信息:")
        for key, values in displacements.items():
            if key != 'time' and values:
                max_val = max(values, key=abs)
                min_val = min(values, key=abs)
                print(f"{key}: 最大值={max_val:.6f}, 最小值={min_val:.6f}, 最大绝对值={max(values, key=abs):.6f}")
        
        return times, displacements
        
    except Exception as e:
        print(f"获取位移数据时出错: {e}")
        import traceback
        traceback.print_exc()
        return None, None

def save_displacement_to_csv(times, displacements, filename):
    """
    保存位移数据到CSV文件
    """
    try:
        # 创建DataFrame
        df = pd.DataFrame(displacements)
        
        # 确保输出目录存在
        output_dir = os.path.dirname(filename)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        # 保存到CSV文件
        df.to_csv(filename, index=False)
        print(f"位移数据已保存到: {filename}")
        
        return True
    except Exception as e:
        print(f"保存CSV文件时出错: {e}")
        return False

def plot_displacement_curves(times, displacements, node_name, save_path=None):
    """
    绘制位移时程曲线
    """
    try:
        # 设置中文字体
        plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
        plt.rcParams['axes.unicode_minus'] = False
        
        # 创建子图
        fig, axes = plt.subplots(3, 2, figsize=(15, 12))
        fig.suptitle(f'节点 {node_name} 位移时程响应', fontsize=16, fontweight='bold')
        
        # 位移和旋转标签
        displacement_keys = ['UX', 'UY', 'UZ']
        rotation_keys = ['RX', 'RY', 'RZ']
        displacement_labels = ['X方向位移 (mm)', 'Y方向位移 (mm)', 'Z方向位移 (mm)']
        rotation_labels = ['X方向旋转 (rad)', 'Y方向旋转 (rad)', 'Z方向旋转 (rad)']
        
        # 绘制位移曲线
        for i, (key, label) in enumerate(zip(displacement_keys, displacement_labels)):
            ax = axes[i, 0]
            if key in displacements and displacements[key]:
                ax.plot(times, displacements[key], 'b-', linewidth=1)
                ax.set_title(label)
                ax.set_xlabel('时间 (s)')
                ax.set_ylabel('位移 (mm)')
                ax.grid(True, alpha=0.3)
        
        # 绘制旋转曲线
        for i, (key, label) in enumerate(zip(rotation_keys, rotation_labels)):
            ax = axes[i, 1]
            if key in displacements and displacements[key]:
                ax.plot(times, displacements[key], 'r-', linewidth=1)
                ax.set_title(label)
                ax.set_xlabel('时间 (s)')
                ax.set_ylabel('旋转 (rad)')
                ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        # 保存图片
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"位移时程曲线图已保存到: {save_path}")
        
        plt.show()
        
    except Exception as e:
        print(f"绘制位移曲线时出错: {e}")

# 主程序执行
def main_analysis():
    print("=" * 60)
    print("重新连接SAP2000并读取节点位移数据")
    print("=" * 60)
    
    # 重新连接到SAP2000
    model = reconnect_to_sap2000()
    if model is None:
        print("连接失败，程序终止")
        return
    
    # 要分析的节点列表（可以修改为您需要的节点）
    target_nodes = ["195", "54000071"]  # 可以添加更多节点
    
    # 荷载工况名称
    load_case = "Wind_time_history"
    
    # 结果存储目录
    results_dir = "D:\\MyFiles\\SAP2000\\results"
    
    all_results = {}
    
    for node_name in target_nodes:
        print(f"\n{'='*40}")
        print(f"分析节点: {node_name}")
        print(f"{'='*40}")
        
        # 获取位移数据
        times, displacements = get_displacement_data(model, node_name, load_case)
        print(f"times: {times}...")
        print(f"displacements: {displacements}...")
        
        if times and displacements:
            # 保存到全局变量以便后续查看
            all_results[node_name] = {
                'times': times,
                'displacements': displacements
            }
            
            # 保存CSV文件
            csv_filename = os.path.join(results_dir, f"node_{node_name}_displacement.csv")
            save_displacement_to_csv(times, displacements, csv_filename)
            
            # 绘制并保存位移曲线
            plot_filename = os.path.join(results_dir, f"node_{node_name}_displacement_curves.png")
            plot_displacement_curves(times, displacements, node_name, plot_filename)
            
        else:
            print(f"节点 {node_name} 位移数据获取失败")
    
    print(f"\n{'='*60}")
    print("分析完成！")
    print(f"{'='*60}")
    
    return all_results

# 执行分析
results = main_analysis()

# 显示results变量的内容
print("\n可用的结果数据:")
for node_name in results.keys():
    print(f"  results['{node_name}']['displacements'] - 节点{node_name}的位移数据")
    print(f"  results['{node_name}']['times'] - 节点{node_name}的时间数据")

重新连接SAP2000并读取节点位移数据
重新连接到SAP2000...
成功重新连接到SAP2000！当前模型文件: D:\MyFiles\SAP2000\1.1model2\1栋-QG.sdb

分析节点: 195

正在获取节点 195 的位移数据...
时程分析包含 16000 个时间步，步长为 0.12015331563074484 秒


KeyboardInterrupt: 

In [9]:
results['195']['times']

[0.0, 0.12015331563074484]